In [3]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d crawford/20-newsgroups

 35% 9.00M/25.7M [00:00<00:00, 67.4MB/s]
100% 25.7M/25.7M [00:00<00:00, 120MB/s] 


In [5]:
! unzip 20-newsgroups.zip

Archive:  20-newsgroups.zip
  inflating: alt.atheism.txt         
  inflating: comp.graphics.txt       
  inflating: comp.os.ms-windows.misc.txt  
  inflating: comp.sys.ibm.pc.hardware.txt  
  inflating: comp.sys.mac.hardware.txt  
  inflating: comp.windows.x.txt      
  inflating: list.csv                
  inflating: misc.forsale.txt        
  inflating: rec.autos.txt           
  inflating: rec.motorcycles.txt     
  inflating: rec.sport.baseball.txt  
  inflating: rec.sport.hockey.txt    
  inflating: sci.crypt.txt           
  inflating: sci.electronics.txt     
  inflating: sci.med.txt             
  inflating: sci.space.txt           
  inflating: soc.religion.christian.txt  
  inflating: talk.politics.guns.txt  
  inflating: talk.politics.mideast.txt  
  inflating: talk.politics.misc.txt  
  inflating: talk.religion.misc.txt  


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from nltk.tokenize import word_tokenize #Used to extract words from documents
from nltk.stem import WordNetLemmatizer #Used to lemmatize words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans

import sys
from time import time

import pandas as pd
import numpy as np
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
from sklearn.datasets import fetch_20newsgroups

'fetch_20newsgroups' is a function in the scikit-learn library (a popular machine learning library in Python) that allows you to load the 20 Newsgroups dataset. The 20 Newsgroups dataset is a collection of approximately 20,000 newsgroup documents across 20 different topics or categories.

In [8]:
categories = [
    'rec.sport.baseball',
    'comp.sys.ibm.pc.hardware',
    'sci.space',
]

print("Loading 20 newsgroups dataset for categories:")
print(categories)

Loading 20 newsgroups dataset for categories:
['rec.sport.baseball', 'comp.sys.ibm.pc.hardware', 'sci.space']


In [11]:
df = fetch_20newsgroups(subset='train', categories=categories,
                             shuffle=False, remove=('headers', 'footers', 'quotes'))

The function fetch_20newsgroups retrieves the dataset and returns a dictionary-like object with the following attributes:

data: A list of strings containing the text of the newsgroup posts.
target: An array of integers representing the category labels for each document.
target_names: A list of strings representing the category names or labels.
DESCR: A description of the dataset.

In [12]:
df.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [13]:
df.data[0]

"\nmorgan and guzman will have era's 1 run higher than last year, and\n the cubs will be idiots and not pitch harkey as much as hibbard.\n castillo won't be good (i think he's a stud pitcher)"

In [14]:
df.target[0] , df.target_names[0] , df.filenames[0]

(1,
 'comp.sys.ibm.pc.hardware',
 '/root/scikit_learn_data/20news_home/20news-bydate-train/rec.sport.baseball/102736')

In [15]:
labels = df.target

In [16]:
len(labels) , len(df.data)

(1780, 1780)

Lemmatizing: Lemmatizing is the process of reducing words to their base or dictionary form, known as the lemma. It involves considering the context and morphological analysis of words to determine their base form. For example, the lemma of "running," "runs," and "ran" is "run."

Lemmatizing limitations:

Lemmatizing requires access to comprehensive linguistic resources or dictionaries.
It may struggle with irregular or domain-specific terms that are not present in the resources.
Lemmatizing can be slower and computationally more intensive compared to stemming.
Stemming limitations:

Stemming relies on simple rules and heuristics, so it may produce base forms that are not actual words (e.g., "comput" instead of "compute").
Stemming does not consider the context or part of speech of the word, which can result in less accurate base forms.
Stemming can sometimes over-stem, collapsing different words with the same root but different meanings (e.g., "play" and "playing" both becoming "play").

In general, lemmatizing is often preferred for document clustering because it retains the linguistic accuracy and context of words. By reducing words to their base forms, lemmatizing helps to group together documents that contain related or similar concepts, even if they use different inflected forms of words

In [17]:
!pip install spacy

In [18]:
!python -m spacy download en

2023-06-28 23:43:52.919312: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 23:43:56.919128: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [19]:
import spacy
from spacy.lang.en import English

# Load the English model in SpaCy
nlp = spacy.load('en_core_web_sm')

# Define a function for lemmatization
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text

# Example usage
for i in range(len(df.data)):
    df.data[i] = lemmatize_text(df.data[i])


In [20]:
print(df.data[0])


 morgan and guzman will have era 's 1 run high than last year , and 
  the cub will be idiot and not pitch harkey as much as hibbard . 
  castillo will not be good ( I think he be a stud pitcher )


In [21]:
vectorizer = TfidfVectorizer(strip_accents='unicode', stop_words='english', min_df=2) ## Corpus is in English
X = vectorizer.fit_transform(df.data)

TF-IDF vectorizer stands for Term Frequency-Inverse Document Frequency vectorizer. It is used to convert text documents into numerical vectors based on the importance of each word in the document and the entire corpus. It assigns higher weights to words that appear frequently in a document but infrequently in the entire corpus, capturing their relative importance. It is commonly used in natural language processing tasks, such as document classification and information retrieval.

Consider the following three documents:

Document 1: "I love cats."
Document 2: "I love dogs."
Document 3: "I hate spiders."

To convert these documents into TF-IDF vectors, we follow these steps:

Term Frequency (TF): Calculate the frequency of each word in each document. For example, in Document 1, the word "I" appears once, "love" appears once, and "cats" appears once.

Inverse Document Frequency (IDF): Calculate the inverse document frequency of each word. This measures how important a word is in the entire corpus. Words that appear frequently across the corpus have lower IDF values, while words that appear in fewer documents have higher IDF values.

TF-IDF Calculation: Multiply the term frequency (TF) of each word in a document by its inverse document frequency (IDF). This calculates the TF-IDF value for each word in each document.

For example, let's assume that the IDF values for "I" is 0.3, "love" is 0.2, "cats" is 0.5, "dogs" is 0.5, and "spiders" is 0.5.

The TF-IDF vectors for the three documents would be:

Document 1: [0.3, 0.2, 0.5, 0, 0]
Document 2: [0.3, 0.2, 0, 0.5, 0]
Document 3: [0.3, 0, 0, 0, 0.5]

In this example, the TF-IDF vector representation captures the importance of each word in the documents. Words that are unique to a document or have higher importance within that document receive higher TF-IDF values. This allows us to compare and analyze the documents based on their textual content using numerical vectors.

In [22]:
X.shape

(1780, 8179)

In [23]:
X[0]

<1x8179 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

# **K_MEANS++**

In [24]:
km = KMeans(n_clusters=3, init='k-means++', max_iter=100)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


done in 0.276s


In [25]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=1000))

Homogeneity: 0.342
Completeness: 0.464
V-measure: 0.394
Adjusted Rand-Index: 0.235
Silhouette Coefficient: 0.012


In [26]:
centroids = km.cluster_centers_.argsort()[:, ::-1] ## Indices of largest centroids' entries in descending order
terms = vectorizer.get_feature_names_out()
for i in range(3):
    print("Cluster %d:" % i, end='')
    for ind in centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: game year team player win run good hit think baseball
Cluster 1: space use know just thank like card think time problem
Cluster 2: drive scsi controller ide disk bus hard floppy mb use


In [29]:
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans

# # Assuming you have your data and labels
# X = df.data
# labels = labels

# # Apply dimensionality reduction using PCA
# pca = PCA(n_components=2)
# X_pca = pca.fit_transform(X)

# # Fit the K-means model
# kmeans = KMeans(n_clusters=3)
# kmeans.fit(X)

# # Plot the clusters
# plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans.labels_, cmap='viridis')
# plt.title("K-means Clustering")
# plt.xlabel("PC1")
# plt.ylabel("PC2")
# plt.show()
